In [1]:
%matplotlib inline

import io, os, sys, types, datetime, pickle, warnings

import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator

import numpy as np
from numpy.linalg import eig, inv

import math

from scipy import interpolate, spatial, stats

import seaborn as sns

import skimage.io as skiIo
from skimage import exposure, img_as_float

from sklearn import linear_model, metrics

from statannot import add_stat_annotation
# https://github.com/webermarcolivier/statannot

warnings.filterwarnings('ignore')

In [2]:
global isMac
isMac = False

if(isMac):
    sup_prefix = '/Volumes/Project/2015_07_Neural_Superposition/Projects/Weiyue/Data_Analysis/Fate_Switching_experiments'
    module_path = '/Volumes/Project/2015_07_Neural_Superposition/Projects/Weiyue/Code/python_cluster/helper_functions'
    sum_fig_out_prefix = '/Users/lily/Desktop/Figure1_sup'
else:
    sup_prefix = 'W:\\2019_09_NSP_Extension\\figure\\analysis\\Fate_Switching_Experiments\\Gal80'
    module_path = 'W:\\2015_07_Neural_Superposition\\Projects\\Weiyue\\code\\python_cluster\\helper_functions\\'
    sum_fig_out_prefix = 'W:\\2019_09_NSP_Extension\\figure\\analysis\\Fate_Switching_experiments\\Figure1'

data_prefix = os.path.join(sup_prefix, 'Data_Gal80')
fig_out_prefix = os.path.join(sup_prefix, 'Output_Gal80', 'FigureOutput_v3')
data_out_prefix = os.path.join(sup_prefix, 'Output_Gal80', 'DataOutput_summary', 'Fz')


In [3]:
module_path

'W:\\2015_07_Neural_Superposition\\Projects\\Weiyue\\code\\python_cluster\\helper_functions\\'

In [4]:
### import custom functions
if module_path not in sys.path:
    sys.path.append(module_path)
import Data_quantification_function_helper as my_help
import Data_quantification_function_intensity_calculation as my_int
import Data_quantification_function_parse_bundle as my_pb
import Data_quantification_function_plotting as my_plot
import Data_quantification_function_summary_helper8 as my_sum

In [5]:
""" ============== Golbal variables and parameters =============="""

' ============== Golbal variables and parameters =============='

In [6]:
### analysis parameters
slicetype = 0
centertype = 1
radius_expanse_ratio = 3
num_angle_ection = 24
num_outside_angle = 18
num_x_section = 40
z_offset = 20
analysis_params = [num_angle_ection, num_outside_angle, num_x_section, z_offset, radius_expanse_ratio]

In [7]:
target_index_match = {0:0, 1:2, 2:3, 3:4, 4:5, 5:7}
target_index_match_rev = {0:0, 2:1, 3:2, 4:3, 5:4, 7:5}
color_code = {1:'#00FFFF', 2:'#1FF509', 3: '#FF0000', 4: '#CFCF1C', 5: '#FF00FF', 6: '#FFAE01', 7:'#ff7c80', 0:'#FFFFFF'}
channel_mapping = {'RFP':0, 'GFP':1, 'R3_1':2, 'R4_1':3, 'R3_2':4, 'R4_2':5, 'R3_3': 6, 0:'RFP', 1:'GFP', 2:'R3_1', 3:'R4_1', 4:'R3_2', 5:'R4_2', 6:'R3_3'}
channel_cmap = {0:'Reds', 1: 'Greens', 2:'Reds', 3: 'Greens', 4:'Reds', 5: 'Greens', 6:'Reds'}
matching_info = (target_index_match, color_code, channel_mapping, channel_cmap, target_index_match_rev)

In [8]:
"""============== main =============="""

'============== main =============='

In [9]:
""" Load data """

' Load data '

In [10]:
### file parameters
# file folders
image_folder = 'Images'
ROI_folder = 'ROIs'
annotation_folder = 'Annotations'
summary_folder = 'Summary'


In [11]:
# load summary statistics file
mypath = os.path.join(data_prefix, summary_folder)
# sum_df_nic = pd.read_csv(os.path.join(mypath, 'summary_v1017_nic.csv'))
sum_df_fz = pd.read_csv(os.path.join(mypath, 'summary_v1017_fz.csv'))

In [12]:
# load annotation files
mypath = os.path.join(data_prefix, annotation_folder, 'Fz')
folders, files = my_help.parseFolderInfo(mypath)
for filename in files:
    print(filename)
    if('.csv' in filename):
        df_temp = pd.read_csv(os.path.join(mypath, filename))
        if(files.index(filename) == 0):
            annots_df = df_temp
        else:
            df_temp = pd.read_csv(os.path.join(mypath, filename))
            annots_df = annots_df.append(df_temp, ignore_index=True, sort=True)
#     print(df_temp)

Fz_Gal80_s26r1_summary.csv
Fz_Gal80_s23r1_summary.csv
Fz_Gal80_s24r1_summary.csv
Fz_Gal80_s38r1_summary.csv
Fz_Gal80_s2r1_summary.csv
Fz_Gal80_s9r1_summary.csv
Fz_Gal80_s1r1_summary.csv


In [13]:
# load data files
outputData = {}
filepaths = my_help.getFilePaths(os.path.join(data_out_prefix))
for i in range(len(filepaths)):
    print(f'====={i}=====')
    pickle_in = open(filepaths[i],"rb")
    outputData[i] = pickle.load(pickle_in)
    category = outputData[i]['categoryID']
    sampleID = outputData[i]['sampleID']
    regionID = outputData[i]['regionID']
    print(category, sampleID, regionID)

=====0=====
Fz 2 1
=====1=====
Fz 24 1
=====2=====
Fz 38 1
=====3=====
Fz 23 1
=====4=====
Fz 9 1
=====5=====
Fz 26 1
=====6=====
Fz 1 1


In [14]:
# outputData[0]['parameter']

In [56]:
import Data_quantification_function_summary_helper12 as my_sum

In [57]:
del bundle_pairs_df

In [58]:
# sum_df = pd.concat([sum_df_nic, sum_df_fz])
sum_df = sum_df_fz
sum_df = sum_df.dropna()
sum_df.reset_index(drop = True, inplace = True)

In [59]:
""" Process sum_df """
## average angles, make angle radian, calculate real length based on positions of heels and targets.
# sum_df_nic = my_sum.process_sum_df(sum_df_nic, annots_df, outputData)
# sum_df_fz = my_sum.process_sum_df(sum_df_fz, annots_df, outputData)
params = ['length_mean_fromheel', 'angle_mean_abs']
sum_df,  bundle_pairs_df = my_sum.process_sum_df(sum_df, annots_df, outputData, params)

===
Fz 2 1
===
Fz 24 1
===
Fz 38 1
===
Fz 23 1
===
Fz 9 1
===
Fz 26 1
===
Fz 1 1


In [60]:
bundle_pairs_df = bundle_pairs_df.dropna()

In [27]:
a = pd.DataFrame(columns = ['R3_length', 'R3_angle', 'R4_length', 'R4_angle', 'type_bundle'])

In [31]:
a.loc[0,['R3_length', 'R3_angle']] = sum_df.loc[0,params].tolist()

In [32]:
a

,R3_length,R3_angle,R4_length,R4_angle,type_bundle
0,1.69541,1.83,NaN,NaN,NaN


In [ ]:
# _, bins = np.histogram(df["variable"])
# g = sns.FacetGrid(df, hue="group")
# g = g.map(sns.distplot, "variable", bins=bins)
# sns.jointplot

In [62]:
len(bundle_pairs_df)

58

In [ ]:
sum_df.loc[:,['angle_max_1', 'bundle_no_total', 'heel_pos_type', 'type_plot']]

In [ ]:
a = pd.DataFrame(columns = ['a', 'b', 'c'])
b = pd.DataFrame(columns = ['a', 'b', 'c'])



In [ ]:
a.loc[1,'a'] = 5
a.loc[5,'c'] = 10
b = pd.concat([a,b], ignore_index = True)

In [ ]:
b

In [ ]:
sum_df_grouped = sum_df.groupby(['type_plot', 'heel_pos_type'])
# sum_df_current = sum_df_grouped.get_group('N')[sum_df_grouped.get_group('N').SampleID != 29] 
sum_df_grouped.groups

In [ ]:
sum_df_grouped.count()

In [ ]:
sns.distplot(sum_df_grouped.get_group(('R3',3.0)).loc[:,'length_mean_fromheel'])
sns.distplot(sum_df_grouped.get_group(('R4',4.0)).loc[:,'length_mean_fromheel'])
sns.distplot(sum_df_grouped.get_group(('R3/R3',3.0)).loc[:,'length_mean_fromheel'])
sns.distplot(sum_df_grouped.get_group(('R3/R3',4.0)).loc[:,'length_mean_fromheel'])
legend = ['R3', 'R4', 'R3(R3/R3)', 'R4(R3/R3)']
plt.xlabel('Relative Length')
plt.legend(legend, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


In [ ]:
sns.distplot(sum_df_grouped.get_group(('R3',3.0)).loc[:,'angle_mean_abs'])
sns.distplot(sum_df_grouped.get_group(('R4',4.0)).loc[:,'angle_mean_abs'])
sns.distplot(sum_df_grouped.get_group(('R3/R3',3.0)).loc[:,'angle_mean_abs'])
sns.distplot(sum_df_grouped.get_group(('R3/R3',4.0)).loc[:,'angle_mean_abs'])
legend = ['R3', 'R4', 'R3(R3/R3)', 'R4(R3/R3)']
plt.xlabel('Relative Angle')
plt.legend(legend, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


In [ ]:
import numpy as np

In [ ]:
np.zeros((1,2))

In [ ]:
R3R3 =np.zeros((1,2))
R3R4 = np.zeros((1,2))

In [ ]:
sum_df.index

In [ ]:
sum_df_grouped = sum_df.groupby(['bundle_no_total'])

In [ ]:
sum_df_grouped.groups.keys()

In [ ]:
sum_df_grouped.get_group(i).iloc[0]['type_plot']

In [ ]:
for i in sum_df_grouped.groups.keys():
    if(len(sum_df_grouped.get_group(i).index) == 2):
        if(sum_df_grouped.get_group(i).iloc[0]['type_plot'] == 'R3'):
            length_R3 = sum_df_grouped.get_group(i).iloc[0]['length_mean_fromheel']
            length_R4 = sum_df_grouped.get_group(i).iloc[1]['length_mean_fromheel']
            R3R4 = np.append(R3R4, [[length_R3, length_R4]], axis = 0)
        elif(sum_df_grouped.get_group(i).iloc[0]['type_plot'] == 'R4'):
            length_R3 = sum_df_grouped.get_group(i).iloc[1]['length_mean_fromheel']
            length_R4 = sum_df_grouped.get_group(i).iloc[0]['length_mean_fromheel']
            R3R4 = np.append(R3R4, [[length_R3, length_R4]], axis = 0)
        elif(sum_df_grouped.get_group(i).iloc[0]['type_plot'] == 'R3/R3'):
            if(sum_df_grouped.get_group(i).iloc[0]['heel_pos_type'] == 3):
                length_R3 = sum_df_grouped.get_group(i).iloc[0]['length_mean_fromheel']
                length_R4 = sum_df_grouped.get_group(i).iloc[1]['length_mean_fromheel']
                R3R3 = np.append(R3R3,  [[length_R3, length_R4]], axis = 0)
            elif(sum_df_grouped.get_group(i).iloc[0]['heel_pos_type'] == 4):
                length_R3 = sum_df_grouped.get_group(i).iloc[1]['length_mean_fromheel']
                length_R4 = sum_df_grouped.get_group(i).iloc[0]['length_mean_fromheel']
                R3R3 = np.append(R3R3,  [[length_R3, length_R4]], axis = 0)

In [ ]:
R3R4 = R3R4[1:,:]
R3R3 = R3R3[1:,:]
x = [0:0.1:3]

In [ ]:
fig = plt.subplots(1,1,figsize = (5,4))
sns.scatterplot(x = R3R4[:,0], y = R3R4[:,1])
sns.scatterplot(x = R3R3[:,0], y = R3R3[:,1])
plt.plot([0,3],[0,3])
plt.xlim([0,3])
plt.ylim([0,3])
legend = ['x=y', 'R3/R4', 'R3/R3']
plt.xlabel('R3 length')
plt.ylabel('R4 length')
plt.legend(legend, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


In [ ]:
len(R3R4)


In [ ]:
len(R3R3)

In [ ]:
np.append(R3R4,[[1,2]], axis = 0)

In [ ]:
[length_R3, length_R4]

In [ ]:
R3R4

In [ ]:
for ind, bundle_no_total in enumerate(sum_df.index):
    print(ind)

In [ ]:
# """ Plotting: summary bar plots """

In [ ]:
# x_cat = "type_plot"
# y_cat = "length_mean_fromheel" 
# y_label = 'Relative Length'
# # 'angle_max_abs','angle_avg_abs'
# # 'length_max_fromheel', 'length_avg_fromheel', 'length_max_fromT3', 'length_avg_fromT3', 

# ylim = (0, 2)
# # ylim = (0, 1.5)

# tickfontsize = 30
# labelfontsize = 40
# figsize = (8,6)

# figure_title = 'length_barplot.png'
# fig_save_path = os.path.join(sum_fig_out_prefix, figure_title)

# sns.set(font_scale=1.5)
# sns.set_style("whitegrid", {'axes.grid' : False})
# x_order = ["R3", "R4", "R4/R4"]
# stats_pair = [('R3', 'R4'), ('R4/R4', 'R4')]

In [ ]:
# sns.set(font_scale=1.5)
# sns.set_style("white")
# fig, ax = plt.subplots(figsize=figsize)
# sns.barplot(ax = ax, x=x_cat, y=y_cat, data=sum_df, color = "silver")
# ax = sns.swarmplot(ax = ax, x=x_cat, y=y_cat, data = sum_df, color=".25", order = x_order)

# ax.set_xlabel("",fontsize=labelfontsize);
# ax.set_ylabel(y_label,fontsize=labelfontsize);
# ax.tick_params(axis = 'x', labelsize = tickfontsize)
# ax.tick_params(axis = 'y', labelsize = tickfontsize)
# ax.set_title("", fontsize = tickfontsize)
# ax.set_ylim(ylim)

# testResults = add_stat_annotation(ax, data = sum_df, x = x_cat, y = y_cat, order = x_order,
#                                   boxPairList=stats_pair, 
#                                   test='Mann-Whitney', textFormat='star', loc='outside', 
#                                   pvalueThresholds=[[1,"ns"], [0.05,"*"], [1e-2,"**"], [1e-3,"***"]], 
#                                   lineOffsetToBox=0.5, lineOffset=0.02, lineHeight=0.03, textOffset=0, verbose=2, 
#                                   fontsize = tickfontsize)
# # test = 't-test_ind'
# plt.savefig(fig_save_path, dpi=300, bbox_inches='tight')
# # p_value_calculation(sum_df, x_cat, y_cat, [('R3', 'R4'), ('R3/R3', 'R3'), ("R4/R4", "R4")])

In [ ]:
""" Plotting: summary bar plots by category """

In [ ]:
sum_df.columns

In [ ]:
sum_df_grouped = sum_df.groupby(['CategoryID'])

In [ ]:
sum_df_current = sum_df_grouped.get_group('Fz')
pert_type = 'fz'
x_order = ["R3", "R4", "R3/R3"] 
test_pair = [('R3', 'R4'), ('R3/R3', 'R3')] 

# sum_df_current = sum_df_grouped.get_group('N')[sum_df_grouped.get_group('N').SampleID != 29] 
# pert_type = 'nic'
# x_order = ["R3", "R4", "R4/R4"]
# test_pair = [('R3', 'R4'), ('R4/R4', 'R4')]

In [ ]:
x_cat = "type_plot"
y_cat = "length_mean_fromheel" 
y_label = 'Relative Length'
hue = 'SampleID'
ylim = (-0.2, 3.5)
fig_title =  f'{pert_type}_length'


# x_cat = "type_plot"
# y_cat = "angle_mean_abs" 
# y_label = 'Relative Angle'
# hue = 'SampleID'
# ylim = (-0.2, 3.2)
# fig_title = f'{pert_type}_angle'


fig_format = 'eps'
fig_name = f'{fig_title}.{fig_format}'

# x_cat = "type_plot"
# y_cat = "length_max_fromheel" 
# y_label = 'Relative Length'
# hue = 'SampleID'




tickfontsize = 30
labelfontsize = 40
figsize = (8,6)

sns.set(font_scale=1.5)
sns.set_style("whitegrid", {'axes.grid' : False})

In [ ]:
fig_name

In [ ]:
sum_fig_out_prefix

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

ax = sns.boxplot(ax = ax, x=x_cat, y=y_cat, data = sum_df_current, color = "silver", order = x_order)
ax = sns.swarmplot(ax = ax, x=x_cat, y=y_cat, data = sum_df_current, color=".25", order = x_order)
# ax = sns.swarmplot(ax = ax, x=x_cat, y=y_cat, data = sum_df_current, hue = hue, order = x_order)

ax.set_xlabel("",fontsize=labelfontsize);
ax.set_ylabel(y_label,fontsize=labelfontsize);
ax.tick_params(axis = 'x', labelsize = tickfontsize)
ax.tick_params(axis = 'y', labelsize = tickfontsize)
ax.set_title("", fontsize = tickfontsize)


testResults = add_stat_annotation(ax, data = sum_df_current, x = x_cat, y = y_cat, order = x_order,
                                  boxPairList=test_pair, 
                                  test='Mann-Whitney', textFormat='star', loc='inside', 
                                  pvalueThresholds=[[1,"ns"], [0.05,"*"], [1e-2,"**"], [1e-3,"***"]], 
                                  lineOffsetToBox=0.1, lineOffset=0.01, lineHeight=0.08, textOffset=1, verbose=2, 
                                  fontsize = tickfontsize)
ax.set_ylim(ylim)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

fig_save_path = os.path.join(sum_fig_out_prefix, fig_name)
plt.savefig(fig_save_path, dpi=1000, bbox_inches='tight', format = fig_format)

my_sum.p_value_calculation(sum_df_current, x_cat, y_cat,test_pair)

In [ ]:
"""Plotting summary polar plot"""

In [ ]:
# pert_cat = 'Fz'
# pert_type = 'R3/R3'
# pert_rtype = 3

pert_cat = 'N'
pert_type = 'R4/R4'
pert_rtype = 4

In [ ]:
theta_cat = 'angle_mean_radian'
r_cat = 'length_mean_fromheel'

In [ ]:
tickfontsize = 30
labelfontsize = 40
figsize = (8,6)
fig_format = 'png'
figure_title = f'{pert_cat}_polar_summary.{fig_format}'
fig_save_path = os.path.join(sum_fig_out_prefix, figure_title)

In [ ]:
sum_df_grouped = sum_df.groupby(['CategoryID', 'type_plot'])

ind_r3 = sum_df_grouped.get_group((pert_cat, 'R3')).index
ind_r4 = sum_df_grouped.get_group((pert_cat, 'R4')).index
r3 = sum_df_grouped.get_group((pert_cat, pert_type)).loc[:,theta_cat] > 0
r4 = sum_df_grouped.get_group((pert_cat, pert_type)).loc[:,theta_cat] < 0
ind_pert_r3 = sum_df_grouped.get_group((pert_cat, pert_type))[r3].index
ind_pert_r4 = sum_df_grouped.get_group((pert_cat, pert_type))[r4].index

pos_t3 = np.mean(sum_df.groupby(['CategoryID']).get_group(pert_cat).loc[:,'pos_t3'].values)
pos_t7 = np.mean(sum_df.groupby(['CategoryID']).get_group(pert_cat).loc[:,'pos_t7'].values)

thetas = {
    'R3':sum_df.loc[ind_r3,theta_cat].values,
    'R4':sum_df.loc[ind_r4,theta_cat].values,
    'pert_R3':sum_df.loc[ind_pert_r3,theta_cat].values,
    'pert_R4':sum_df.loc[ind_pert_r4,theta_cat].values,
}
rs = {
    'R3':sum_df.loc[ind_r3,r_cat].values,
    'R4':sum_df.loc[ind_r4,r_cat].values,
    'pert_R3':sum_df.loc[ind_pert_r3,r_cat].values,
    'pert_R4':sum_df.loc[ind_pert_r4,r_cat].values,
}


plot_line = {
    'R3':'-',
    'R4':'-',
    'pert_R3':'--',
    'pert_R4':'--',
}
plot_color = {
    'R3':color_code[3],
    'R4':color_code[4],
    'pert_R3':color_code[pert_rtype],
    'pert_R4':color_code[pert_rtype],
}

targetGridPolar = my_sum.getTargetGridPolar_summary(pos_t3, pos_t7)
legend = ['R3', 'R4', pert_type]



In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0.1, 0.1, 0.75, 0.79], polar=True)
for i in thetas.keys():
    ax.errorbar(np.mean(thetas[i]), np.mean(rs[i]), xerr = np.std(thetas[i]), yerr = np.std(rs[i]), color = 'k', elinewidth=2)
    ax.plot([0,np.mean(thetas[i])], [0,np.mean(rs[i])], linewidth = 5, linestyle = plot_line[i], color = plot_color[i])

# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 8, box.height])
for i in [0,2,3,5]:
    ax.plot(targetGridPolar[i,0], targetGridPolar[i,1], 'o', color = color_code[target_index_match[i]], markersize = 20, mew = 3, mfc = 'none')

# Put a legend to the bottom left of the current axis
ax.legend(legend, loc='lower left', bbox_to_anchor=(-0.1, 0.7), fontsize = 30)

ax.set_thetamin(-30)
ax.set_thetamax(30)
ax.tick_params(axis = 'x', labelsize = 30)
ax.tick_params(axis = 'y', labelsize = 30)

# plt.show()
plt.savefig(fig_save_path, dpi=1000, bbox_inches='tight', format = fig_format)


In [ ]:
""" Summary of polar density map """

In [ ]:
matrices = my_sum.get_summary_matrices(sum_df, annots_df, outputData)

In [ ]:
pert_cat = 'Fz'
bundle_type = 'R3R4'
channel_no = 1
figsize = (10,8)
colormap = plt.get_cmap('gray')
z_min = -1
z_max = 0.15 # default is -1

pos_t3 = np.mean(sum_df.groupby(['CategoryID']).get_group(pert_cat).loc[:,'pos_t3'].values)
pos_t7 = np.mean(sum_df.groupby(['CategoryID']).get_group(pert_cat).loc[:,'pos_t7'].values)

figure_params = figsize, colormap, z_min, z_max

calculation_params = analysis_params, radius_expanse_ratio, pos_t3, pos_t7

matrix = matrices[category][bundle_type][:,channel_no,:,:,:]
plot_matrix = np.mean(np.mean(matrix, axis = 3), axis = 0)

fig = my_sum.plot_summary_polar(plot_matrix, channel_no, calculation_params, matching_info, figure_params)
fig_save_path = os.path.join(sum_fig_out_prefix, figure_title)
plt.savefig(fig_save_path, dpi=300, bbox_inches='tight')

plt.show()



In [ ]:
def plot_summary_polar(matrix, channelNo, calculation_params, matching_info, figure_params):
#     vmax_set = 1
    analysisParams, radiusExpanseRatio, pos_t3, pos_t7 = calculation_params
    figsize, colormap, z_min, z_max, cbar_tick = figure_params
    
    targetIndexMatch, ColorCode, channel_mapping, channel_cmap, targetIndexMatch_rev = matching_info
    
#     colormap = plt.get_cmap(channel_cmap[channelNo])
    # colormap = plt.get_cmap('gray')
    
    thetav, rv, z1, norm1, targetGridPolar = my_sum.getPolarPlotValues(analysisParams, channelNo, matrix, colormap, targetIndexMatch_rev, radiusExpanseRatio, pos_t3, pos_t7, z_min, z_max)
    
    fig = plt.figure(figsize = figsize)
#     fig = plt.figure()
    ax2 = fig.add_subplot(111, polar = True)

    ## plot value
    sc = ax2.pcolormesh(thetav, rv, z1, cmap=colormap, norm=norm1)

    ## plot heel position
    for i in [2,3,5]:
        ax2.plot(targetGridPolar[i,0], targetGridPolar[i,1], 'o', color = ColorCode[targetIndexMatch[i]], markersize = 20, mew = 5, mfc = 'none')
    
    ## plot angle reference
    ax2.plot([0, targetGridPolar[targetIndexMatch_rev[3],0]], [0, targetGridPolar[targetIndexMatch_rev[3],1]], '--', color = '0.5')
    ax2.plot([0, targetGridPolar[targetIndexMatch_rev[7],0]], [0, targetGridPolar[targetIndexMatch_rev[7],1]], '--', color = '0.5')

    ## set polar to pie
    ax2.set_thetamin(-40)
    ax2.set_thetamax(40)
    
    ax2.set_rlim(0, 3)
    # ax.set_xticklabels({'fontsize':20})
    ax2.tick_params(axis = 'x', labelsize = 30)
    ax2.tick_params(axis = 'y', labelsize = 30)

    #### color bar for polar plot
    vmin,vmax = sc.get_clim()
#     vmax = 0.2
    cNorm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)   #-- Defining a normalised scale
    ax5 = fig.add_axes([0.8, 0.2, 0.02, 0.6])       #-- Creating a new axes at the right side
    cb1 = matplotlib.colorbar.ColorbarBase(ax5, norm=cNorm, cmap=colormap, ticks=cbar_tick)    #-- Plotting the colormap in the created axes
    cb1.ax.tick_params(labelsize=30)
    fig.subplots_adjust(left=0.0,right=0.95)
    
    return fig

In [ ]:
""" Individual Density Maps """

In [ ]:
for iData in outputData.keys():
    category = outputData[iData]['categoryID']
    sampleID = outputData[iData]['sampleID']
    regionID = outputData[iData]['regionID']
    print(iData, category, sampleID, regionID)

In [ ]:
outputData[iData].keys()

In [ ]:
import Data_quantification_function_summary_helper5 as my_sum

In [ ]:
iData = 5
bundle_no = 6 #1, 19
 
category = outputData[iData]['categoryID']
sampleID = outputData[iData]['sampleID']
regionID = outputData[iData]['regionID']

In [ ]:
# channel_no = 0
# z_min = 0
# z_max = 0.25
# r_lim = (0, 3)

channel_no = 1
z_min = 0
z_max = 0.2
r_lim = (0, 2.5)

In [ ]:
figsize = (10,8)
colormap = plt.get_cmap('gray')


cbar_tick = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
tickfontsize = 30
labelfontsize = 40
fig_format = 'eps'
fig_name = f'{category}_s{sampleID}r{regionID}_bd_no{bundle_no}_c{channel_no}.{fig_format}'
fig_save_path = os.path.join(sum_fig_out_prefix, fig_name)

In [ ]:
matrix.shape

In [ ]:
category = outputData[iData]['categoryID']
sampleID = outputData[iData]['sampleID']
regionID = outputData[iData]['regionID']
rel_pos = outputData[iData]['relativePositions']

sum_df_current = sum_df.groupby(['CategoryID', 'SampleID', 'RegionID']).get_group((category, sampleID, regionID))
annots_df_current = annots_df.groupby(['CategoryID', 'SampleID', 'RegionID']).get_group((category, sampleID, regionID))
annots_df_current.loc[:,'Bundle_No'] = annots_df_current.loc[:,'Bundle_No'].values.astype(int)
annots_df_current.set_index('Bundle_No', inplace = True)
annots_df_current.sort_index(axis = 0, inplace = True)

ind_annot = list(annots_df_current.index).index(bundle_no)
ind_sum = sum_df_current.index[(sum_df_current['bundle_no'] == bundle_no)]

matrix = outputData[iData]['intensityMatrix'][ind_annot, channel_no, :,:,:]
plot_matrix = np.mean(matrix, axis = 2)

pos_t3 = rel_pos[ind_annot][2]
pos_t7 = rel_pos[ind_annot][5]

calculation_params = analysis_params, radius_expanse_ratio, pos_t3, pos_t7

figure_params = figsize, colormap, z_min, z_max, r_lim, cbar_tick

fig = my_sum.plot_summary_polar(plot_matrix, channel_no, calculation_params, matching_info, figure_params)

plt.savefig(fig_save_path, dpi=300, bbox_inches='tight', format = fig_format)

plt.show()
